```{index} single: solver; cbc
```
```{index} pandas dataframe
```
```{index} sample average approximation
```
```{index} stochastic optimization
```
```{index} chance constraints
```

# Economic dispatch in energy systems

In [ ]:
# install Pyomo and solvers
import requests
import types

url = "https://raw.githubusercontent.com/mobook/MO-book/main/python/helper.py"
helper = types.ModuleType("helper")
exec(requests.get(url).content, helper.__dict__)

helper.install_pyomo()
helper.install_cbc()

In [ ]:
import pyomo.environ as pyo
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Problem description: Economic dispatch
The _economic dispatch (ED)_ problem is the short-term determination of the optimal production of energy to meet all energy demand. Let $V$ denote a set of nodes, each of which is representing cities, industrial districts, or power generators or combinations of these. Each node $i \in V$ **may** have:
- a certain energy demand $d_i \geq 0$; 
- a power generator whose energy production needs to be between $p_i^{min}$ and $p_i^{max}$ units of power. The cost of producting one unit of power at node $i$ is given by a variable cost $c_i \geq 0$. 

Not all the nodes have both demand and generation, more specifically it is possible for a node to have only generation or only demand.

The goal is to determine for each node $i \in V$ the optimal production level $p_i$, such that the total energy demand is met, no production limits are exceeded, and the total energy production costs are minimized. We can formulate the problem as the following MILP:

$$
\begin{align}
\begin{array}{llll}
\min        & \sum_{i \in V} c_i p_i\\
\text{s.t.} & \sum_{i \in V} p_i = \sum_{i \in V} d_i,\\
& p_{i}^{min} \leq p_{i} \leq p_{i}^{max} & \forall i \in V.
\end{array}
\end{align}
$$

Now assume that we have built several off-shore wind turbines. These wind turbines combined together produce a random nonnegative amount of extra energy, denoted by $\omega$.

$$
\begin{align}
\begin{array}{llll}
\min        & \sum_{i \in V} c_i p_i\\
\text{s.t.} & \omega + \sum_{i \in V} p_i = \sum_{i \in V} d_i,\\
& p_{i}^{min} \leq p_{i} \leq p_{i}^{max} & \forall i \in V.
\end{array}
\end{align}
$$

Because of stochastic fluctuations in wind power generation, the ED problem is best modeled as a stochastic optimization problem. The intermittency of wind generation makes it almost impossible to perfectly balance supply and demand, but in practice there is some room for error. We denote by
- $\Delta \geq 0$ the tolerance of the absolute power difference between supply and demand;
- $\varepsilon \in [0,1]$ is the risk level we are willing to accept for the supply to deviate from the demand more than $\Delta$;
- $\omega \in \mathbb{R}_{\geq 0}$ the nonnegative random variable describing the total power production of off-shore wind turbines.

Instead of requiring that the supply and demand are matched perfectly, we require that the absolute difference remains below power threshold $\Delta$ using the following chance constraint:

$$
\begin{align}
  \mathbb{P} \Big ( \Big | \omega + \sum_{i \in V } p_i  - \sum_{i \in V} d_i \Big | \leq \Delta \Big) \geq 1 - \varepsilon
\end{align}
$$

Breaking this up (hence "relaxing" it) into two individual chance constraints, this leads to the following optimization problem with chance constraints:

$$
\begin{align}
\begin{array}{llll}
\min        & \sum_{i \in V } c_i(p_i)\\
\text{s.t.} & \mathbb{P}(\omega + \sum_{i \in V } p_i  - \sum_{i \in V} d_i \leq \Delta) \geq 1 - ɛ\\
& \mathbb{P}(\omega + \sum_{i \in V } p_i  - \sum_{i \in V} d_i \geq -\Delta) \geq 1 - ɛ\\
& p_{i}^{min } \leq p_{i} \leq p_{i}^{max } & \forall i \in V.
\end{array}
\end{align}
$$

The goal of this notebook is to implement this chance-constrained model.

In [ ]:
def read_economic_dispatch_data():
    nodes_df = pd.read_csv(
        "nodes.csv",
        index_col=0,
    )[["node_id", "d", "p_min", "p_max", "c_var"]]

    wind_production_samples_df = pd.read_csv(
        "discrete_wind.csv"
    ).T

    # Read data
    nodes = nodes_df.set_index("node_id").T.to_dict()
    wind_production_samples = list(wind_production_samples_df.to_dict().values())
    wind_production_samples = [sum(d.values()) for d in wind_production_samples]

    return nodes, wind_production_samples

nodes, wind_production_samples = read_economic_dispatch_data()

The `nodes` dictionary contains for every $i \in V$ information about $p_i^{min}$, $p_i^{max}$, $c_i$, $d_i$. Although not relevant for this assignment, there is a clear distinction in the data between nodes that only consume power, and nodes that only produce power.

In [ ]:
nodes[0] # first node properties

The wind production samples can be accessed through the `wind_production_samples` variable. It is a list of 500 equiprobable outcomes for the wind generation.

In [ ]:
wind_production_samples[4] # fifth outcome

We estimate the wind power production using historical data. You are given 500 outcomes of the total wind producton ```wind_production_samples```, all equiprobable (some outcomes are repeated, but you can treat them as different if you stick with the equal probability assumption).

We rewrite the chance-constrained ED problem as a mixed-integer linear program:

\begin{align}
\begin{array}{llll}
\min        & \sum_{i \in V} c_i(p_i)\\
\text{s.t.} & \omega_j + \sum_{i \in V} p_i  - \sum_{i \in V} d_i \leq \Delta + u_jM_j & \forall j = 1, \dots, N\\
&  \omega_j + \sum_{i \in V} p_i  - \sum_{i \in V} d_i \geq -\Delta - u_jM_j & \forall j = 1, \dots, N\\
& \sum_{j=1}^{N}u_j \leq \varepsilon N \\
& p_{i}^{min } \leq p_{i} \leq p_{i}^{max } & \forall i \in V, 
\end{array}
\end{align}

In words, we introduce a new binary variable $u_j$ for each sample $j=1, \dots, N$. For each sample, the supply and demand constraints are deactivated when $u_j =1$ and $u_j=0$ otherwise. Note that we only use one single $u_j$ variable for each constraint. Having two separate $u^{(1)}_j$ and $u^{(2)}_j$ will yield the same objective value, but the model is incorrect w.r.t. the violation of the supply and demand constraint introduced earlier. The $M_j$'s here should be selected based on the data. A reasonable choice for $M_j$ is to take it equal to the left-hand side minus $\Delta$ while replacing $p_i$ for $p_i^{max}$.

In [ ]:
def economic_dispatch(nodes, samples, eps, Delta):
    # Define a model
    model = pyo.ConcreteModel("Q1")

    model.N = pyo.Set(initialize=range(len(samples)))
    model.nodes = pyo.Set(initialize=nodes.keys())

    M = {j: (samples[j] +
          sum(data['p_max'] for _, data in nodes.items()) +
          sum(data['d'] for _, data in nodes.items()) - Delta)
          for j in range(len(samples))}

    # Declare decision variables
    model.p = pyo.Var(nodes, domain=pyo.NonNegativeReals)
    model.u = pyo.Var(model.N, domain=pyo.Binary)

    # Declare objective value
    @model.Objective(sense=pyo.minimize)
    def objective(m):
        return sum(data["c_var"] * m.p[i] for i, data in nodes.items())


    @model.Constraint(model.N)
    def supply_demand_leq(m, j):
        wind = samples[j]
        supply = sum(m.p[i] for i, data in nodes.items())
        demand = sum(data['d'] for _, data in nodes.items())
        return wind + supply - demand  <= Delta + M[j] * m.u[j]

    @model.Constraint(model.N)
    def supply_demand_geq(m, j):
        wind = samples[j]
        supply = sum(m.p[i] for i, data in nodes.items())
        demand = sum(data['d'] for _, data in nodes.items())
        return wind + supply - demand  >= -Delta - M[j] * m.u[j]

    @model.Constraint()
    def success_probability(m):
        return sum(m.u[j] for j in model.N) <= eps * len(model.N)

    @model.Constraint(model.nodes)
    def generation_upper_bound(m, i):
        return m.p[i] <= nodes[i]["p_max"]

    @model.Constraint(model.nodes)
    def generation_lower_bound(m, i):
        return nodes[i]["p_min"] <= m.p[i]

    return model

We now solve the model for the provided instance and wind production outcomes and report the optimal objective value you obtain for $\varepsilon = 0.02$ and $\Delta=1000$.

In [ ]:
# Data
eps = 0.20
Delta = 1000
N = 500
argMin = int(N*eps)

# Solve model
model = economic_dispatch(nodes, wind_production_samples, eps, Delta)
result = gurobi_solver.solve(model)

# The production is equal to the total demand - Delta - smallest_wind_production
sum_production = sum([model.p[x].value for x in model.p])
sum_demand = sum(data['d'] for i, data in nodes.items())
smallest_wind_production =  sorted([wind_production_samples[j] for j in range(N)])[argMin]

print("Total demand:", sum_demand)
print("Delta:", Delta)
print("Smallest wind production", smallest_wind_production)
print("Model solution of total production:", sum_production)

# The total production should be produced by the cheapest generators.
print("Analytical solution of total production:", sum_demand - Delta - smallest_wind_production)

We now solve the same MILP varying the values first of $\varepsilon \in [0, 1]$ (for fixed $\Delta=1000$) and then of $\Delta \in [0, 2000]$ (for fixed $\varepsilon = 0.02$).     

In [ ]:
fixed_Delta = 1000

feas_eps = []
feas_objs = []

for eps in np.linspace(0, 1, num=20):
    model = economic_dispatch(nodes, wind_production_samples, eps, fixed_Delta)
    result = gurobi_solver.solve(model)

    if result.solver.termination_condition == 'optimal':
        feas_eps.append(eps)
        feas_objs.append(model.objective())

plt.plot(feas_eps, feas_objs, marker='o', linestyle='--')
plt.xlabel('epsilon')
plt.ylabel('objective value') 

In [ ]:
fixed_eps = 0.02

feas_Deltas = []
feas_objs = []

for Delta in np.linspace(0, 2000, num=20):
    model = economic_dispatch(nodes, wind_production_samples, fixed_eps, Delta)
    result = gurobi_solver.solve(model)

    if result.solver.termination_condition == 'optimal':
        feas_Deltas.append(Delta)
        feas_objs.append(model.objective())

plt.plot(feas_Deltas, feas_objs, marker='o', linestyle='--')
plt.xlabel('$\Delta$')
plt.ylabel('objective value') 

We can make the following observations:

- Smaller values $\varepsilon$ and $\Delta$ lead to more infeasibilities, which is to be expected. Smaller $\varepsilon$ allow for less constraint violations/relaxations, whereas smaller Delta make constraints tighter (and thus easier to violate).
- The reason why the plot becomes flat for high $\varepsilon$ and $\Delta$ values is because production is no longer needed. For instance, a high $\varepsilon$ means we can ignore the $N\varepsilon$ worst-case sample scenarios, whereas higher $\Delta$ means that we do not need to produce to match demand.
- The model can be solved analytically.
  - The (deterministic) Economic Dispatch problem is always solved by producing using the cheapest generators. Therefore, any optimal solution will always use the cheapest generators possible.
  - It will never violate the supply-demand balance by overproduction, since producing less will always lead to cheaper solutions.
  - By adding chance constraints where the outcomes follow a equiprobable discrete distribution, we know that scenarios with low wind production will be violated before the scenarios with large wind production. 
  - Given $\varepsilon$ and $\Delta$, what will happen is that the $N\varepsilon$ (rounded down) samples with lowest wind production will be deactivated.
  - The exact total production will be $$\sum_{i \in V} p_i = \sum_{i \in V}d_i - \Delta - \Omega$$
  where $\Omega$ is the $N\varepsilon$-th sample with smallest wind production. The $p_i$ can be determined by selecting the cheapest generators first.